In [2]:
pip install --upgrade xgboost

In [2]:
!pip install whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.8 MB/s eta 0:00:00


In [4]:
# 👇 Make sure this path is correct
import pandas as pd
import requests
import whois
from datetime import datetime
from bs4 import BeautifulSoup

data = pd.read_csv(r'/content/drive/MyDrive/final.csv')
data.columns = data.columns.str.strip()

print("✅ Loaded Columns:")
print(data.columns.tolist())

✅ Loaded Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks',

In [1]:
# Install if needed
!pip install pandas numpy scikit-learn xgboost imbalanced-learn python-whois beautifulsoup4 requests

# Import
import pandas as pd
import numpy as np
import requests
import whois
from urllib.parse import urlparse
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import joblib


In [5]:
# 1️⃣ Install tqdm if needed
!pip install tqdm

# 2️⃣ Imports
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import requests
import whois
from datetime import datetime
from bs4 import BeautifulSoup

# 3️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 4️⃣ Load CSV from Drive
data = pd.read_csv(r
                   '/content/drive/MyDrive/final.csv')
data.columns = data.columns.str.strip()

print("✅ Loaded Columns:")
print(data.columns.tolist())

# 5️⃣ Define the missing function
def get_html_keyword_count(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        keywords = soup.find_all('meta', attrs={'name': 'keywords'})
        return len(keywords)
    except Exception as e:
        return 0

# 6️⃣ Apply with progress bar
print("\n✅ Adding html_keyword_count (this may take time)...")
data['html_keyword_count'] = data['url'].progress_apply(get_html_keyword_count)
print("✅ html_keyword_count added!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path

100%|██████████| 5000/5000 [1:08:57<00:00,  1.21it/s]

✅ html_keyword_count added!


In [6]:
print(data[['url', 'html_keyword_count']].head(10))

                                                 url  html_keyword_count
0                https://www.rga.com/about/workplace                   0
1                             http://starasia.com.hk                   1
2        https://www.youtube.com/watch?v=XszqITK-UEw                   1
3                         http://www.civico1845.com/                   0
4  http://rapidpaws.com/wp-content/we_transfer/in...                   0
5             https://mykoreankitchen.com/dak-galbi/                   0
6  http://support-appleld.com.secureupdate.duilaw...                   0
7                               http://quiluc.com.vn                   1
8  http://www.astro.com/astrology/tma_article1607...                   1
9         https://huggybearheadcanon.tumblr.com/#_=_                   0


In [7]:
# ------------------------------------------
# ✅ Define suspicious words for blacklist
# ------------------------------------------
BLACKLIST_KEYWORDS = [
    "login", "verify", "account", "secure",
    "update", "bank", "password"
]

# ------------------------------------------
# ✅ Function to check if URL is "blacklisted"
# ------------------------------------------
def check_blacklist_flag(url):
    try:
        url = str(url).lower()
        for word in BLACKLIST_KEYWORDS:
            if word in url:
                return 1
        return 0
    except Exception:
        return 0

# ------------------------------------------
# ✅ Add the blacklist_flag column with progress bar
# ------------------------------------------
from tqdm import tqdm
tqdm.pandas()

print("\n✅ Adding blacklist_flag column (this is fast)...")
data['blacklist_flag'] = data['url'].progress_apply(check_blacklist_flag)
print("✅ blacklist_flag column added!")

# ------------------------------------------
# ✅ Preview to confirm
# ------------------------------------------
print(data[['url', 'blacklist_flag']].head(10))
data.to_csv('/content/dataset_phishing_augmented.csv', index=False)
print("✅ CSV with new features saved!")




✅ Adding blacklist_flag column (this is fast)...


100%|██████████| 5000/5000 [00:00<00:00, 116371.38it/s]

✅ blacklist_flag column added!
                                                 url  blacklist_flag
0                https://www.rga.com/about/workplace               0
1                             http://starasia.com.hk               0
2        https://www.youtube.com/watch?v=XszqITK-UEw               0
3                         http://www.civico1845.com/               0
4  http://rapidpaws.com/wp-content/we_transfer/in...               0
5             https://mykoreankitchen.com/dak-galbi/               0
6  http://support-appleld.com.secureupdate.duilaw...               1
7                               http://quiluc.com.vn               0
8  http://www.astro.com/astrology/tma_article1607...               0
9         https://huggybearheadcanon.tumblr.com/#_=_               0


✅ CSV with new features saved!


In [8]:
data = pd.read_csv(r'/content/drive/MyDrive/dataset_phishing_augmented (1).csv')
TARGET_COLUMN = 'status'  # Or your real target
X = data.drop(TARGET_COLUMN, axis=1)
y = data[TARGET_COLUMN]

In [9]:
!pip install pandas requests beautifulsoup4

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [11]:
# 👇 Make sure this path is correct
data = pd.read_csv(r'/content/drive/MyDrive/dataset_phishing_augmented (1).csv')
data.columns = data.columns.str.strip()

print("✅ Loaded Columns:")
print(data.columns.tolist())

✅ Loaded Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks',

In [12]:
TARGET_COLUMN = 'status'   # ✅ this is your actual label

# Drop label and the raw 'url' column from features
X = data.drop([TARGET_COLUMN, 'url'], axis=1)
y = data[TARGET_COLUMN]

print("\n✅ X shape:", X.shape)
print("✅ y distribution:\n", y.value_counts())


✅ X shape: (5000, 89)
✅ y distribution:
 status
phishing      2513
legitimate    2487
Name: count, dtype: int64


In [13]:
print("\n✅ Checking for missing values:")
print(X.isnull().sum())

# Fill any NaNs with 0
X = X.fillna(0)
print("✅ Missing values filled with 0.")


✅ Checking for missing values:
length_url            0
length_hostname       0
ip                    0
nb_dots               0
nb_hyphens            0
                     ..
dns_record            0
google_index          0
page_rank             0
html_keyword_count    0
blacklist_flag        0
Length: 89, dtype: int64
✅ Missing values filled with 0.


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n✅ Train/Test Split:")
print("Train samples:", X_train.shape[0])
print("Test samples :", X_test.shape[0])
X_train = X_train.values
X_test = X_test.values


✅ Train/Test Split:
Train samples: 4000
Test samples : 1000


check class Balance

In [15]:
print("\n✅ Class balance before SMOTE:")
print(pd.Series(y_train).value_counts())


✅ Class balance before SMOTE:
status
phishing      2010
legitimate    1990
Name: count, dtype: int64


apply smote for balancing

In [16]:
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

print("\n✅ After SMOTE balancing:")
print(pd.Series(y_train_bal).value_counts())


✅ After SMOTE balancing:
status
legitimate    2010
phishing      2010
Name: count, dtype: int64


In [17]:
param_dist = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 3, 5]
}

setup xgboost classifier

In [18]:
xgb_model = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    verbosity=0
)

Setup RandomizedSearchCV

In [19]:
search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

n Hyperparameter Tuning

In [20]:
# ✅ Convert label to numeric (safely)
data['status'] = data['status'].replace({
    'legitimate': 0,
    'phishing': 1
}).astype(int)

# ✅ Define features and target
TARGET_COLUMN = 'status'
X = data.drop([TARGET_COLUMN, 'url'], axis=1)
y = data[TARGET_COLUMN]

print("\n✅ X shape:", X.shape)
print("✅ y distribution:\n", y.value_counts())


✅ X shape: (5000, 89)
✅ y distribution:
 status
1    2513
0    2487
Name: count, dtype: int64


/tmp/ipython-input-20-3866681060.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['status'] = data['status'].replace({


In [22]:
# ===========================================
# ✅ 1️⃣ Import Libraries
# ===========================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import xgboost as xgb

# ===========================================
# ✅ 2️⃣ Load Data
# ===========================================
data = pd.read_csv('/content/dataset_phishing_augmented.csv')  # Change path if needed
print("\n✅ Original Columns:")
print(data.columns.tolist())

# ===========================================
# ✅ 3️⃣ Label Conversion
# ===========================================
data['status'] = data['status'].replace({
    'legitimate': 0,
    'phishing': 1
})
print("\n✅ Label Conversion Done")
print(data['status'].value_counts())

# ===========================================
# ✅ 4️⃣ Define Features and Target
# ===========================================
TARGET_COLUMN = 'status'
X = data.drop([TARGET_COLUMN, 'url'], axis=1)
y = data[TARGET_COLUMN]

print("\n✅ X Shape:", X.shape)
print("✅ y distribution:\n", y.value_counts())

# ===========================================
# ✅ 5️⃣ Train-Test Split
# ===========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("\n✅ Train-Test Split Done")
print(f"Train Samples: {X_train.shape[0]}")
print(f"Test Samples : {X_test.shape[0]}")

# ===========================================
# ✅ 6️⃣ Balance Training Data with SMOTE
# ===========================================
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
print("\n✅ After SMOTE Balancing:")
print(y_train_bal.value_counts())

# ===========================================
# ✅ 7️⃣ Define XGBoost Model
# ===========================================
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

# ===========================================
# ✅ 8️⃣ Define Parameter Grid
# ===========================================
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150, 200],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    scoring='accuracy',
    n_iter=20,
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# ===========================================
# ✅ 9️⃣ Hyperparameter Tuning (NO early stopping)
# ===========================================
print("\n✅ Starting Hyperparameter Tuning (no early stopping in search)...")
search.fit(X_train_bal, y_train_bal)
print("\n✅ Hyperparameter Tuning Complete!")

print("\n✅ Best Hyperparameters Found:")
print(search.best_params_)

# ===========================================
# ✅ 🔟 Refit Best Model with Early Stopping
# ===========================================
print("\n✅ Refitting best model with early stopping...")
best_params = search.best_params_
final_model = xgb.XGBClassifier(
    **best_params,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

# ✅ Older-version-compatible early stopping:
final_model.fit(
    X_train_bal, y_train_bal,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=20,
    verbose=False
)

# ===========================================
# ✅ 1️⃣1️⃣ Evaluate Model
# ===========================================
preds_train = final_model.predict(X_train)
preds_test = final_model.predict(X_test)

train_acc = accuracy_score(y_train, preds_train)
test_acc = accuracy_score(y_test, preds_test)

print("\n✅ Final Accuracy Results:")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")

print("\n✅ Classification Report:")
print(classification_report(y_test, preds_test))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_test, preds_test))



✅ Original Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlink

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [23]:
import xgboost
print(xgboost.__version__)


3.0.2


In [25]:
# ===========================================
# ✅ 1️⃣ Import Libraries
# ===========================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import xgboost as xgb

# ===========================================
# ✅ 2️⃣ Load Data
# ===========================================
data = pd.read_csv('/content/dataset_phishing_augmented.csv')  # Change path if needed
print("\n✅ Original Columns:")
print(data.columns.tolist())

# ===========================================
# ✅ 3️⃣ Label Conversion
# ===========================================
data['status'] = data['status'].replace({
    'legitimate': 0,
    'phishing': 1
})
print("\n✅ Label Conversion Done")
print(data['status'].value_counts())

# ===========================================
# ✅ 4️⃣ Define Features and Target
# ===========================================
TARGET_COLUMN = 'status'
X = data.drop([TARGET_COLUMN, 'url'], axis=1)
y = data[TARGET_COLUMN]

print("\n✅ X Shape:", X.shape)
print("✅ y distribution:\n", y.value_counts())

# ===========================================
# ✅ 5️⃣ Train-Test Split
# ===========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("\n✅ Train-Test Split Done")
print(f"Train Samples: {X_train.shape[0]}")
print(f"Test Samples : {X_test.shape[0]}")

# ===========================================
# ✅ 6️⃣ Balance Training Data with SMOTE
# ===========================================
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
print("\n✅ After SMOTE Balancing:")
print(y_train_bal.value_counts())

# ===========================================
# ✅ 7️⃣ Define XGBoost Model
# ===========================================
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

# ===========================================
# ✅ 8️⃣ Define Parameter Grid
# ===========================================
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150, 200],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    scoring='accuracy',
    n_iter=20,
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# ===========================================
# ✅ 9️⃣ Hyperparameter Tuning (NO early stopping)
# ===========================================
print("\n✅ Starting Hyperparameter Tuning (no early stopping in search)...")
search.fit(X_train_bal, y_train_bal)
print("\n✅ Hyperparameter Tuning Complete!")

print("\n✅ Best Hyperparameters Found:")
print(search.best_params_)

# ===========================================
# ✅ 🔟 Refit Best Model with Early Stopping (using callback)
# ===========================================
print("\n✅ Refitting best model with early stopping using callback...")
best_params = search.best_params_

final_model = xgb.XGBClassifier(
    **best_params,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

# ✅ Define EarlyStopping callback (safe in 3.0.2)
early_stopping_cb = xgb.callback.EarlyStopping(rounds=20)

final_model.fit(
    X_train_bal, y_train_bal,
    eval_set=[(X_test, y_test)],
    callbacks=[early_stopping_cb],
    verbose=False
)

# ===========================================
# ✅ 1️⃣1️⃣ Evaluate Model
# ===========================================
preds_train = final_model.predict(X_train)
preds_test = final_model.predict(X_test)

train_acc = accuracy_score(y_train, preds_train)
test_acc = accuracy_score(y_test, preds_test)

print("\n✅ Final Accuracy Results:")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")

print("\n✅ Classification Report:")
print(classification_report(y_test, preds_test))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_test, preds_test))



✅ Original Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlink

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'callbacks'

In [26]:
# ===========================================
# ✅ 1️⃣ Import Libraries
# ===========================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

import xgboost as xgb

# ===========================================
# ✅ 2️⃣ Load Data
# ===========================================
data = pd.read_csv('/content/dataset_phishing_augmented.csv')  # Change path if needed
print("\n✅ Original Columns:")
print(data.columns.tolist())

# ===========================================
# ✅ 3️⃣ Label Conversion
# ===========================================
data['status'] = data['status'].replace({
    'legitimate': 0,
    'phishing': 1
})
print("\n✅ Label Conversion Done")
print(data['status'].value_counts())

# ===========================================
# ✅ 4️⃣ Define Features and Target
# ===========================================
TARGET_COLUMN = 'status'
X = data.drop([TARGET_COLUMN, 'url'], axis=1)
y = data[TARGET_COLUMN]

print("\n✅ X Shape:", X.shape)
print("✅ y distribution:\n", y.value_counts())

# ===========================================
# ✅ 5️⃣ Train-Test Split
# ===========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("\n✅ Train-Test Split Done")
print(f"Train Samples: {X_train.shape[0]}")
print(f"Test Samples : {X_test.shape[0]}")

# ===========================================
# ✅ 6️⃣ Balance Training Data with SMOTE
# ===========================================
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
print("\n✅ After SMOTE Balancing:")
print(y_train_bal.value_counts())

# ===========================================
# ✅ 7️⃣ Define XGBoost Model
# ===========================================
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

# ===========================================
# ✅ 8️⃣ Define Parameter Grid
# ===========================================
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150, 200],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    scoring='accuracy',
    n_iter=20,
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# ===========================================
# ✅ 9️⃣ Hyperparameter Tuning (NO early stopping)
# ===========================================
print("\n✅ Starting Hyperparameter Tuning (no early stopping in search)...")
search.fit(X_train_bal, y_train_bal)
print("\n✅ Hyperparameter Tuning Complete!")

print("\n✅ Best Hyperparameters Found:")
print(search.best_params_)

# ===========================================
# ✅ 🔟 Refit Best Model WITHOUT Early Stopping
# ===========================================
print("\n✅ Refitting best model without early stopping...")
best_params = search.best_params_

final_model = xgb.XGBClassifier(
    **best_params,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    verbosity=0,
    random_state=42
)

final_model.fit(X_train_bal, y_train_bal)

# ===========================================
# ✅ 1️⃣1️⃣ Evaluate Model
# ===========================================
preds_train = final_model.predict(X_train)
preds_test = final_model.predict(X_test)

train_acc = accuracy_score(y_train, preds_train)
test_acc = accuracy_score(y_test, preds_test)

print("\n✅ Final Accuracy Results:")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")

print("\n✅ Classification Report:")
print(classification_report(y_test, preds_test))

print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_test, preds_test))



✅ Original Columns:
['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlink

In [27]:
import joblib

joblib.dump(final_model, 'xgb_phishing_model.pkl')


['xgb_phishing_model.pkl']